In [ ]:
%load_ext autoreload
%autoreload 2
from brian2 import *
from multiprocessing import Pool
import numpy as np
import experiment_7_full as ex7
import power_spectral_density as psd

In [ ]:
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('lines', linewidth=3)
plt.rc('legend', fontsize=14)

In [ ]:
def call_ex7_d(d):
    return ex7.run_experiment(n, a, b, c, d)

In [ ]:

n = 1000
a, b, c = 400, 30,8 
p = Pool(6)
#func = lambda d: ex7.run_experiment(n, a, b, c, d)
args_d = np.linspace(0, 4, 6)
results_d = p.map(call_ex7_d, args_d)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(15,6))
dt, shift = 200, 10
for i, data in enumerate(results_d):
    N, X = data['N_EX'], data['X']
    plot_ma(N, X, dt, shift, ax=ax, color="C{}".format(i),
            label="IN_IN_FACTOR: {:.2f}".format(args_d[i]))
ax.legend()
ax.set_title('Varying the IN_IN_FACTOR. Default values are 400, 30, 8, *')
plt.savefig('figures/exp_7_high_ininfactor.png')
plt.show()

In [ ]:
a, b, c, d = 400, 15, 8, 2
data = ex7.run_experiment(1000, a, b, c, d)

In [ ]:
# MA -> yaxis show percentage of all neurons spiking per ms
N, N_EX,X, Y, X2, Y2 = [data2[x] for x in ['N', 'N_EX', 'X', 'Y', 'X2', 'Y2']]
N_IN = N - N_EX
print("{:,} exc spikes, {:,} inh spikes".format(len(X), len(X2)))

end = 4000
mask = np.logical_and.reduce((X >= 1000, X < end))
mask2 = np.logical_and.reduce((X2 >= 1000, X2 < end))

fig, axarr = plt.subplots(3, figsize=(15,15))
axarr[0].plot(X[mask], Y[mask], '.', color='C0', label='Excitatory Population')
axarr[0].plot(X2[mask2], Y2[mask2]+N_EX, '.', color='C1', label='Inhibitory Population')
axarr[0].set_ylabel('Neuron index')
axarr[0].set_xlabel('Simulation Time (ms)')
axarr[0].set_title('Raster plot of spikes')
axarr[0].legend(loc=3)

dt, shift = 5, 5
plot_ma(N_EX, X[mask], dt, shift, ax=axarr[1] , label='Excitatory Population')
plot_ma(N_IN, X2[mask2], dt, shift, ax=axarr[1], label='Inhibitory Population')
axarr[1].set_xlabel('Simulation Time (ms)')
axarr[1].legend()

#fig, axarr = plt.subplots(2, sharex=True, figsize=(15,6))
plot_spectrum(X, dt, shift, ax=axarr[2], label='Excitatory Population')
plot_spectrum(X2, dt, shift, ax=axarr[2], label='Inhibitory Population')
axarr[2].set_xticks(np.arange(0, 100, 5))
axarr[2].legend()

plt.tight_layout()
plt.savefig('figures/exp_7_full_dynamics_2.png')
plt.show()